In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from scipy.stats import norm, uniform

import fig_settings as fs
from src import *

In [ ]:
colors = ["#5f6366", "#3768d2", "#005b7a", "#a6cebd"]
fs.set_fonts()
idx1 = 0
idx2 = 1
idx3 = 2

In [ ]:
n = 100
alpha = 1e-7

training_time = 300  # training time/
testing_time = 10
frac = 0.5
dt = 0.01

w_in_sigma = 1e-3

gateR = 2.7e4
gateC = 8.98e-7
pinchoff = -0.6

parameters = dict()
parameters["transconductance"] = {"mean": 0.582e-3, "stddev": 0.0582e-3}
parameters["channel-width"] = {"mean": 200e-6, "stddev": 0}
parameters["channel-length"] = {"mean": 101e-6, "stddev": 0}
parameters["pinchoff-voltage"] = {"mean": pinchoff, "stddev": 0}  # pinch-off voltage
parameters["weighting-resistor"] = {"mean": 500, "stddev": 100}
parameters["gate-capacitance"] = {"mean": gateC, "stddev": 0.1 * gateC}
parameters["gate-resistance"] = {"mean": gateR, "stddev": 0.1 * gateR}
parameters["applied-drain-voltage"] = {"mean": -0.05, "stddev": 0}

# system
D = 3
r_dist = uniform(100, 500)
delta_dist = norm(scale=0.1)
p = 6 / n
# plist = np.logspace(-3, 0, 16)
# p = plist[3]
sigma = 10
rho = 28
beta = 8 / 3

In [ ]:
u0list = generate_initial_conditions(
    10,
    [-7.4, -11.1, 20],
    delta_dist,
    10,
    1e-4,
    lorenz,
    sigma=10,
    rho=28,
    beta=8 / 3,
)

u0 = u0list[0]

In [ ]:
# OECT parameters
Vdinit, R, Rg, Cg, Vp, Kp, W, L = generate_OECT_parameters(n, parameters)

A = erdos_renyi_network(n, p, r_dist)

w_in = input_layer(D, n, w_in_sigma)

In [ ]:
nx.is_connected(nx.Graph(A))

In [ ]:
nx.draw(nx.Graph(A), node_size=10, node_color="black")

In [ ]:
w_out, uf, r0, V1_0 = train_oect_reservoir(
    u0.copy(),
    training_time,
    dt,
    frac,
    w_in,
    A,
    alpha,
    Vdinit,
    R,
    Rg,
    Cg,
    Vp,
    Kp,
    W,
    L,
    lorenz,
    sigma=sigma,
    rho=rho,
    beta=beta,
)

In [ ]:
t, signal_oect, prediction_oect = run_oect_reservoir_autonomously(
    uf,
    r0,
    V1_0,
    testing_time,
    dt,
    w_in,
    w_out,
    A,
    Vdinit,
    R,
    Rg,
    Cg,
    Vp,
    Kp,
    W,
    L,
    lorenz,
    sigma=sigma,
    rho=rho,
    beta=beta,
)

In [ ]:
w_out, uf, r = train_reservoir(
    u0.copy(),
    w_in,
    A,
    training_time,
    dt,
    frac,
    alpha,
    0,
    lorenz,
    sigma=sigma,
    rho=rho,
    beta=beta,
)

In [ ]:
## Run reservoir autonomously.
t, signal_tanh, prediction_tanh = run_reservoir_autonomously(
    uf,
    r,
    w_in,
    w_out,
    A,
    testing_time,
    dt,
    0,
    lorenz,
    sigma=sigma,
    rho=rho,
    beta=beta,
)